# **Introducción a Python**
## FP27. Decoradores (Decorators) 

    
Los decoradores son un tópico avazado de Python


## <font color=‘blue’>**¿Por qué queremos decoradores?**</font> 

Imagina la siguiente función:

In [ ]:
def mi_func():
    print("Hola Mundo")

In [ ]:
mi_func()

Hola Mundo


Qué pasaría si quisiéramos agregarle más funcionalidad a esta función, en lugar de solo la *print('Hola Mundo')*? 

Por ejemplo, si quisiéramos imprimir 'HOLA MUNDO', todo en mayúsculas.

Hasta ahora, con lo que sabemos, tendríamos que reescribir la función para añadirle algo.

Un **decorador** es un *patrón de diseño* en Python que permite al usuario agregar nueva funcionalidad a un objeto existente sin modificar su estructura.

In [ ]:
def mi_func():
    # Anadir más funcionalidad aquí
    # como por ejemplo otra cláusula print()
    
    # Aquí tenemos la función original
    print("Hola Mundo")
    
    # Incluso puede agregar más funcionalidad después de las operaciones originales

Para completar las áreas comentadas arriba en la función *mi_func*, podemos usar el operador **\@** para adjuntar un decorador. Sin embargo, necesitaremos crear nuestros propios decoradores. Aquí es donde el tema se torna más avanzado!!

Los decoradores se pueden considerar como funciones que modifican la *funcionalidad* de otra función. Ayudan a hacer tu código más corto y más "pythonista".

Para explicar adecuadamente qué es un decorador, construiremos uno paso a paso a partir de funciones. 


**Importante**: Asegúrate de reiniciar el Kernel de este Notebook para que esta lección se vea igual en tu computador. 

### La función

Las funciones en Python son *ciudadanos de primera clas*e. Esto significa que admiten operaciones como pasarlas como argumento a otra función, devolverlas como resultados desde una función, modificarlas y asignarlas a una variable. Este es un concepto fundamental a entender antes de profundizar en la creación de decoradores de Python.

In [ ]:
def func():
    return 1

In [ ]:
func()

1

### Su alcance

Recuerda de la lección FP21 de declaraciones anidadas que Python usa *Scope* para saber a qué se refiere una etiqueta. Por ejemplo:

In [ ]:
s = 'Variable Global'

def func():
    print(f'Mi espacio de nombres local es {locals()}')

Recuerda también, que las funciones de Python crean un nuevo alcance (*scope*), lo que significa que la función tiene su propio espacio de nombres para buscar nombres de variables cuando se mencionan dentro de la función. Podemos verificar variables locales y variables globales con las funciones **local( )** y **globals( )**. 

Por ejemplo:

In [ ]:
# Con esta instrucción verás todas las variables globales que hay en el 
# espacio de nombres del Kernel actual de Python en el cual se ejecuta el presente notebook

print(globals())

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def func():\n    return 1', 'func()', "s = 'Variable Global'\n\ndef func():\n    print(f'Mi espacio de nombres local es {locals()}')", '# Con esta instrucción verás todas las variables globales que hay en el \n# espacio de nombres del Kernel actual de Python en el cual se ejecuta el presente notebook\n\nprint(globals())'], '_oh': {2: 1}, '_dh': ['/content'], '_sh': <module 'IPython.core.shadowns' from '/usr/local/lib/python3.7/dist-packages/IPython/core/shadowns.py'>, 'In': ['', 'def func():\n    return 1', 'func()', "s = 'Variable Global'\n\ndef func():\n    print(f'Mi espacio de nombres local es {locals()}')", '# Con esta instrucción verás todas las variables globales que hay en el \n# espacio de nombres del Kernel ac

In [ ]:
# Veamos el tipo de salida que nos entrega?

type(globals())

dict

Aquí obtenemos un diccionario de todas las variables globales, muchas de ellas predefinidas en Python. Así que sigamos adelante y miremos las claves:

In [ ]:
print(globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', '_sh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'func', '_i2', '_2', '_i3', 's', '_i4', '_i5', '_5', '_i6'])


Fíjatee cómo **s** está ahí, la Variable Global que definimos como una cadena:

In [ ]:
# Veamos qué valor tiene la llave 's' en el diccionario 'globals()'

globals()['s']

'Variable Global'

Ahora ejecutemos nuestra función para verificar si hay variables locales en func().
No debería haber ninguna, por que no definimos ninguna en la función.

In [ ]:
func()

Mi espacio de nombres local es {}


¡Excelente!<br>
Lo anterior fue un preámbulo para entender lo que sigue.<br>
Ahora continuemos con la construcción de la lógica de lo que es un decorador. Recuerda que en Python **todo es un objeto**. Eso significa que las funciones son objetos a los que se les pueden asignar etiquetas y pasarlas como argumetos a otras funciones. 

Comencemos con algunos ejemplos simples:

## Asignar funciones a variables

Para comenzar, creamos una función que agregará uno a un número cada vez que se llame. Luego asignaremos la función a una variable y usaremos esta variable para llamar a la función.

Verás que aquí no estamos usando paréntesis porque no estamos llamando a la función *mas_uno*, sino que simplemente estamos colocando la función en la variable *suma_uno*.
Si incluyéramos paréntesis, en *suma_uno* guardaríamos el valor de salida de la función *mas_uno*!!

In [ ]:
def mas_uno(numero=0):
    return numero + 1

# La función 'mas_uno' es asignada a la variable 'suma_uno'
suma_uno = mas_uno
suma_uno(5)

6

In [ ]:
# Las funciones no quedan atadas. Borraremos mas_uno() y suma_uno() seguirá existiendo

del mas_uno #eliminar variable
suma_uno(3)

4

### Definir funciones dentro de otras funciones

A continuación, ilustraremos cómo puedes definir una función dentro de otra función en Python. 

No te pierdas. Ya veremos cómo todo esto es relevante para crear y comprender decoradores en Python.

In [ ]:
def mas_uno(numero):
    # La función 'suma_uno' está definida dentro de la función 'mas_uno'
    def suma_uno(numero):
        return numero + 1


    resultado = suma_uno(numero)
    return resultado

mas_uno(4)

5

### Pasar funciones como argumentos a otras funciones

Las funciones también se pueden pasar como parámetros a otras funciones. Veamos esto a continuación.

In [ ]:
def mas_uno(numero):
    return numero + 1

def llama_funcion(funcion):
    numero_a_sumar = 5
    return funcion(numero_a_sumar)

# La función 'mas_uno' pasa como argumento de la función 'llama_funcion'
llama_funcion(mas_uno)

6

### Funciones anidadas tienen acceso al *scope* de la función adjunta

Python permite que una función anidada acceda al ámbito (*scope*) externo de la función adjunta. Este es un concepto crítico en los decoradores: este patrón se conoce como **Closure** (Cierre).

In [ ]:
def imprime_mensaje(mensaje):
    """
    Esta es la Función Adjunta
    """
    def mensajero():
        """
        Esta es la Función Anidada
        """
        # La variable 'mensaje' está en el espacio de nombres (Scope) interno de la función
        # 'imprime_mensaje' y la función 'mensajero' puede usarla
        print(mensaje)

    mensajero()

imprime_mensaje("Hola mundo !!")

Hola mundo !!


## <font color=‘blue’>**Creando decoradores**</font> 

Vistos estos conceptos, sigamos adelante y creemos un decorador simple que convertirá una oración en mayúsculas (nuestro objetivo inicial, recuerdan?). 

Haremos esto definiendo un contenedor dentro de una función cerrada. Como ves en la celda siguiente, es muy similar a la función dentro de otra función que creamos anteriormente.

In [ ]:
def decorador_mayusculas(funcion):
    def f_envoltura():                     # Definimos una función dento de otra función
        func = funcion()                   # Asignamos una función a una variable y
                                           # accedemos al 'name space' de la función adjunta
        en_mayusculas = func.upper()
        return en_mayusculas

    return f_envoltura

Nuestra función decoradora (**decorador_mayusculas**) toma una función como argumento (**funcion**) y, por lo tanto, definiremos una función y se la pasaremos a nuestro decorador. Aprendimos antes que podíamos asignar una función a una variable. Usaremos ese truco para llamar a nuestra función decoradora.

In [ ]:
def mi_func():
    return f"Hola Mundo"
    

decorate = decorador_mayusculas(mi_func)
decorate()

'HOLA MUNDO'

Por último, Python nos ofrece una forma mucho más sencilla para aplicar decoradores. Simplemente usamos el símbolo @ antes de la función que nos gustaría decorar. Veamos cómo:


In [ ]:
@decorador_mayusculas
def mi_func():
    return f"Hola Mundo"

mi_func()

'HOLA MUNDO'

Creamos otro decorador, esta vez uno para dividir un *strig* de palabras en una lista de palabras separadas.

In [ ]:
def divide_string(funcion):
    def wrapper():                      # Por convención llamaremos a las funciones internas 'wrappers'
        func = funcion()
        string_dividido = func.split()
        return string_dividido

    return wrapper

In [ ]:
@divide_string
@decorador_mayusculas
def mi_func():
    return f"Hola Mundo"

mi_func()

['HOLA', 'MUNDO']

## Creando un decoradorpara medir tiempo de ejecución

Intentemos recrear un decorador similar al que vimos en el notebook de **Debugging**. Recuerdan?

In [ ]:
def mide_tiempo(func):
    import time

    def wrapper(n):
        t0 = time.time()
        f = func(n)
        t1 = time.time()
        tt = (t1 - t0) 
        print(f'Tiempo = {tt: 1.3f} segundos.')
        return f
    return wrapper

In [ ]:
@mide_tiempo
def suma_cuadrados(n):
    result = [x**2 for x in range(n)]
    return f'La suma de cuadrados de números es: {sum(result)}'

In [ ]:
suma_cuadrados(10000000)

Tiempo =  3.467 segundos.


'La suma de cuadrados de números es: 333333283333335000000'

In [ ]:
# Podemos decorar muchas funciones distintas
@mide_tiempo
def suma_numeros(n):
    result = [x for x in range(n)]
    return f'La suma de números es: {sum(result)}'

In [ ]:
suma_numeros(10000000)

Tiempo =  0.832 segundos.


'La suma de números es: 49999995000000'

## <font color='green'>Actividad 1: Challenging</font> 
### Crea un decorador tipo *@timeit*
A partir del ejemplo anterior, crea un decorador que simule uno del tipo **@timeit**, el cual tomará el tiempo promedio de 7 ejecuciones de la función pasada como argumento.

Nombra tu **decorador** como *tiempo_itera*

In [1]:
def tiempo_itera(func):
    import time
    each_rep = []

    def wrapper(n):
        for i in range(7): #Correr 7 veces función
            t0 = time.time()
            f = func(n)
            t1 = time.time()
            each_rep.append(t1 - t0)
        print(f'{n} loops, best of 7: {min(each_rep): 1.3f} s per loop') #Imprimir tiempo promedio
        return f
    return wrapper

In [2]:
@tiempo_itera
def suma(n): #Función suma de cuadrados
    result = [x**2 for x in range(n)]
    return f'La suma de números al cuadrado es: {sum(result)}'

In [3]:
suma(10000000)

10000000 loops, best of 7:  3.186 s per loop


'La suma de números al cuadrado es: 333333283333335000000'

In [4]:
#Solución alternativa con timeit
%%timeit -r 7
def suma(n):
    result = [x**2 for x in range(n)]
    return f'La suma de números al cuadrado es: {sum(result)}'

suma(10000000)   

1 loop, best of 7: 3.24 s per loop


<font color='orange'> Se pueden crear funciones propias para medir tiempo, pero las librerías de Python tienen funciones potentes

<font color='green'> Fin actividad 1</font>

Genial Hackers !!!